In [1]:
import geopandas as gpd
from os.path import join
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df = gpd.read_file('../../data/Crops/metadata.geojson')

In [3]:
df.head()

,Fold,id,ID_PATCH,N_Parcel,Parcel_Cover,TILE,dates-S2,geometry
0,1,lat46024long-416,t30tyr-0,55,0.867065,t30tyr,"{'0': '20190903', '1': '20190906', '2': '20190...","POLYGON ((-0.41645 46.02437, -0.39993 46.02399..."
1,2,lat46023long-399,t30tyr-1,45,0.854065,t30tyr,"{'0': '20190903', '1': '20190906', '2': '20190...","POLYGON ((-0.39993 46.02399, -0.38341 46.02361..."
2,3,lat46023long-383,t30tyr-2,53,0.816467,t30tyr,"{'0': '20190903', '1': '20190906', '2': '20190...","POLYGON ((-0.38341 46.02361, -0.36689 46.02323..."
3,4,lat46023long-366,t30tyr-3,36,0.598450,t30tyr,"{'0': '20190903', '1': '20190906', '2': '20190...","POLYGON ((-0.36689 46.02323, -0.35037 46.02285..."
4,5,lat46022long-350,t30tyr-4,25,0.561584,t30tyr,"{'0': '20190903', '1': '20190906', '2': '20190...","POLYGON ((-0.35037 46.02285, -0.33385 46.02247..."


In [4]:
def check_cloud_level(mask, prob_thresh=5, img_thresh=0.95):
    return (mask <= 5).mean() >= img_thresh

def filer_cloudy_data(row):
    filename = '/extra/Sentinel-2-crops/{TILE}/{id}/'.format(TILE=row['TILE'].upper(), id=row['id'])
    out = {}
    for _, _date in row['dates-S2'].items():
        cloud = np.load(join(filename, 'cloudProb/{}.npy'.format(_date))).sum(axis=0)
        imags = np.load(join(filename, '{}.npy'.format(_date)))[:3].max()
        if check_cloud_level(cloud) and imags > 0: 
            out.update({str(len(out)):_date})
    return out

In [5]:
df['dates-S2'] = df.parallel_apply(filer_cloudy_data, axis=1)
df = df[df['dates-S2'].apply(len).sort_values() > 30]

/projects/Deepleey/agri-tech/sentinel-datasets/LIPS/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [6]:
df.to_file('../../data/Crops/metadata.geojson')

In [7]:
df

,Fold,id,ID_PATCH,N_Parcel,Parcel_Cover,TILE,dates-S2,geometry
0,1,lat46024long-416,t30tyr-0,55,0.867065,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.41645 46.02437, -0.39993 46.02399..."
2,3,lat46023long-383,t30tyr-2,53,0.816467,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.38341 46.02361, -0.36689 46.02323..."
3,4,lat46023long-366,t30tyr-3,36,0.598450,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.36689 46.02323, -0.35037 46.02285..."
4,5,lat46022long-350,t30tyr-4,25,0.561584,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.35037 46.02285, -0.33385 46.02247..."
5,1,lat46022long-333,t30tyr-5,28,0.753906,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.33385 46.02247, -0.31733 46.02208..."
...,...,...,...,...,...,...,...,...
20396,2,lat43290long1781,t31tcj-7051,64,0.808228,t31tcj,"{'0': '20190903', '1': '20190913', '2': '20191...","POLYGON ((1.78112 43.29035, 1.79690 43.29052, ..."
20397,3,lat43290long1796,t31tcj-7052,57,0.701660,t31tcj,"{'0': '20190903', '1': '20190913', '2': '20191...","POLYGON ((1.79690 43.29052, 1.81267 43.29068, ..."
20398,4,lat43290long1812,t31tcj-7053,58,0.635315,t31tcj,"{'0': '20190903', '1': '20190913', '2': '20191...","POLYGON ((1.81267 43.29068, 1.82845 43.29085, ..."
20399,5,lat43290long1828,t31tcj-7054,54,0.631531,t31tcj,"{'0': '20190903', '1': '20190913', '2': '20191...","POLYGON ((1.82845 43.29085, 1.84422 43.29101, ..."
